# Lesson 16 Support Vector Machines

SVM algo is an optimization algorithm that aims to maximize the margin of a dataset and finding the separating hyperplane that neatly divides the data.  

You can see from these three images below, what that looks like in practice across multiple modes of SVM classification. 

<img src="linear_svm.png" width="250">  
<img src="polynomial_svm.png" width="250">
<img src="radial_basis_func_svm.png" width="250">

# DataSet 
The dataset includes metrics of information on temperature, humidity, pressure, and weather. This can be used to predict energy consumption.  

# Time series  
We will be solving a time-series problem! Here future values depend on past values. 

# Partitioning the Dataset   
Normally, we would do the train-test-split scheme that we have done previously, but this time we are going to be sensitive to the temporal nature of this data. Notice there is a 'date' column at first. Therefore, our split will focus on how much data came before a certain date (**training**) and how much comes after (**testing**). 

# Import

In [2]:
%cd ..

/Users/jaimemerizalde/Desktop/Library/Machine Learning Udemy Course


In [18]:
from Library import data

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

# Get Data

In [8]:
df = data.get_data("Datasets/Energy.csv", index_col=0)
df.head()

,date,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,...,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
0,11-01-2016 17:00,60,30,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,...,17.033333,45.53,6.60,733.5,92.0,7.000000,63.000000,5.3,13.275433,13.275433
1,11-01-2016 17:10,60,30,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,...,17.066667,45.56,6.48,733.6,92.0,6.666667,59.166667,5.2,18.606195,18.606195
2,11-01-2016 17:20,50,30,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,...,17.000000,45.50,6.37,733.7,92.0,6.333333,55.333333,5.1,28.642668,28.642668
3,11-01-2016 17:30,50,40,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,...,17.000000,45.40,6.25,733.8,92.0,6.000000,51.500000,5.0,45.410390,45.410390
4,11-01-2016 17:40,60,40,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,...,17.000000,45.40,6.13,733.9,92.0,5.666667,47.666667,4.9,10.084097,10.084097


# EDA

We have all umerical data it seems. The <mark> target variable we are intersted in is "appliances" </mark> because it is actually the column tracking energy us in Wh.

In [ ]:
pd.set_option("display.max_columns", None) # Displays all columns in pandas dataframe
df.head()

print(f"Total rows: {len(df)}\nTotal columns (including output var): {df.shape[1]}")

df.describe(include="all")

Total rows: 19735
Total columns (including output var): 29


0    60
1    60
2    50
3    50
4    60
Name: Appliances, dtype: int64

# Partition Dataset

In [27]:
from sklearn.model_selection import train_test_split

y = df["Appliances"]
X = df[[c for c in df.columns if c != "Appliances"]]

# See my note about this above.
# X_train, X_test, y_train, y_test = train_test_split()
#
print(f"Proportion split of training-to-test data: {12000 / X.shape[0]}.")
X_train = X.iloc[:12000]
X_test = X.iloc[12000:]
y_train = y.iloc[:12000] 
y_test = y.iloc[12000:]

Proportion split of training-to-test data: 0.6080567519635166.


(7735,)